In [1]:
# import external packages
import numpy as np
import cvxpy as cp
import mosek
from sklearn.model_selection import train_test_split
import time

# import internal packages
import phi_divergence as phi
from iter_gen_and_eval_alg import iter_gen_and_eval_alg

The problem we examine is as follows:

Consider a company that sells $n$ different products, which it is able to produce with the use of $m$ different machines. 

The goal is to determine an optimal production plan, which specifies the amount of time $y_{jk}$ that each machine $j=1,\dots,m$ will be used for producing product $k = 1,\dotsc,n$. An optimal plan is one that maximizes the total profit of the company subject to availability constraints.

Each machine $j$ may only be used for a limited amount of time $a_j$ and incurs operating costs $c_{jk}$ per unit of product $k$ that is produced. Each unit of product $k$ can be sold at a price of $u_k$ and leftover units incur inventory holding costs of $\tilde{c}_{k}$. For this problem there are two uncertain parameters, the demand $d_k$ for each product $k$ and the quantity $p_{jk}$ of product $k$ that is produced per time unit by machine $j$. 

Putting all this together, we arrive at the following mathematical formulation:
\begin{align}
    \label{mathform:weighted_dist_1:obj}
    \min_{\mathbf{y}}&~\sum_{j=1}^{m} \sum_{k=1}^{n} c_{jk} y_{jk} + \sum_{k=1}^{n} \tilde{c}_{k} \left[ \sum_{j=1}^{m} p_{jk} y_{jk} - d_k \right]_{+} - \sum_{k=1}^{n} u_k \min \left( \sum_{j=1}^{m} p_{jk} y_{jk}, d_k \right) \\
    \text{s.t.}&~\sum_{k=1}^{n} y_{jk} \leq a_j, && j=1.\dotsc,m \label{mathform:weighted_dist_1:limit_time}\\
    &~y_{jk} \geq 0, && j=1.\dotsc,m, k=1.\dotsc,n, \label{mathform:weighted_dist_1:xVarNonNeg}
\end{align}


where $[ a ]_{+}$ is equivalent to $\max\{a,0\}$. The $\max$ and $\min$ functions make this a nonlinear optimization problem. As such, it is quite a tricky problem to deal with using conventional Robust Optimization techniques. 

Note that this problem can also be easily converted into the notation used earlier for by moving the uncertainty into the constraints and introducing supplementary variable $\theta$.
Let $\mathbf{x} = (\mathbf{y}, \theta)$ and $\mathscr{X} = \{ \mathbf{x} : \sum_{k=1}^{n} y_{jk} \leq a_j, j=1.\dotsc,m, \mathbf{y} \geq 0 \}$. We can rewrite the problem as:
\begin{align}
    \label{mathform:weighted_dist_2:obj}
    \min_{\mathbf{x} \in \mathscr{X}}&~\sum_{j=1}^{m} \sum_{k=1}^{n} c_{jk} y_{jk} + \theta  \\
    \text{s.t.}&~\sum_{k=1}^{n} \tilde{c}_{k} \left[ \sum_{j=1}^{m} p_{jk} y_{jk} - d_k \right]_{+} - \sum_{k=1}^{n} u_k \min \left( \sum_{j=1}^{m} p_{jk} y_{jk}, d_k \right) - \theta \leq 0.
\end{align}

By defining $g(\mathbf{x}) = \sum_{j=1}^{m} \sum_{k=1}^{n} c_{jk} y_{jk} + \theta$ and $f(\mathbf{x}, \mathbf{z}) = \sum_{k=1}^{n} \tilde{c}_{k} \left[ \sum_{j=1}^{m} p_{jk} y_{jk} - d_k \right]_{+} - \sum_{k=1}^{n} u_k \min \left( \sum_{j=1}^{m} p_{jk} y_{jk}, d_k \right) - \theta$ we obtain the familiar notation used throughout this paper. 

In [2]:
# problem specific functions:
def generate_unc_param_data(random_seed, N, **kwargs):
    np.random.seed(random_seed)
    m = 5
    n = 10
    
    # generate demand vector param
    d_nom = (25, 38, 18, 39, 60, 35, 41, 22, 74, 30)
    d = np.random.default_rng().dirichlet(d_nom, N) * 382
    
    # generate production efficiency param
    p_nom = np.array([[5.0, 7.6, 3.6, 7.8, 12.0, 7.0, 8.2, 4.4, 14.8, 6.0],
                      [3.8, 5.8, 2.8, 6.0, 9.2, 5.4, 6.3, 3.4, 11.4, 4.6],
                      [2.3, 3.5, 1.6, 3.5, 5.5, 3.2, 3.7, 2.0, 6.7, 2.7],
                      [2.6, 4.0, 1.9, 4.1, 6.3, 3.7, 4.3, 2.3, 7.8, 3.2],
                      [2.4, 3.6, 1.7, 3.7, 5.7, 3.3, 3.9, 2.1, 7.0, 2.9]])
    p = np.random.random_sample(size = (N,m,n)) * (p_nom*1.05 - p_nom*0.95) + (p_nom*0.95)
    data = list(zip(d,p))
    return data

def get_fixed_param_data():
    # fixed parameter values from Care (2014)
    C = np.array([[1.8, 2.2, 1.5, 2.2, 2.6, 2.1, 2.2, 1.7, 2.8, 1.9],
                  [1.6, 1.9, 1.3, 1.9, 2.3, 1.9, 2.0, 1.5, 2.5, 1.7],
                  [1.2, 1.5, 1.0, 1.5, 1.9, 1.4, 1.6, 1.1, 2.0, 1.3],
                  [1.3, 1.6, 1.1, 1.6, 2.0, 1.5, 1.7, 1.2, 2.2, 1.4],
                  [1.2, 1.5, 1.0, 1.6, 1.9, 1.5, 1.6, 1.1, 2.1, 1.3]])

    A = np.array([10, 13, 22, 19, 21])
    C_tilde = np.array([1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3])
    U = np.array([1.5, 1.8, 1.2, 1.9, 2.2, 1.8, 1.9, 1.4, 2.4, 1.6])
    param_dict = {'C':C, 'A':A, 'C_tilde': C_tilde, 'U': U}
    return param_dict

def solve_P_SCP(S, **kwargs):
    # get fixed parameter values
    C = kwargs['C']
    A = kwargs['A']
    C_tilde = kwargs['C_tilde']
    U = kwargs['U']
    
    # unzip uncertain parameters
    d,p = zip(*S)
    
    # get dimensions of problem
    m,n = p[0].shape
    num_scen = len(d)
    
    # create variables
    theta = cp.Variable(1)
    y = cp.Variable((m, n), nonneg = True)
    
    # set up problem
    constraints = []
    for s in range(num_scen):
        unc_inv_cost_s = sum(C_tilde[k] * cp.maximum(sum(p[s][j][k]*y[j][k] for j in range(m)) - d[s][k], 0)
                             for k in range(n))
        unc_rev_s = sum(U[k] * cp.minimum(sum(p[s][j][k]*y[j][k] for j in range(m)),d[s][k]) 
                        for k in range(n))
        constraints.append(unc_inv_cost_s - unc_rev_s - theta <= 0)
    
    constraints.append(cp.sum(y, axis=1) <= A)
    
    fixed_costs = cp.sum(cp.multiply(C, y))
    obj = cp.Minimize(fixed_costs + theta)
    prob = cp.Problem(obj,constraints)
    
    # solve problem
    time_limit = kwargs.get('time_limit', 2*60*60)    
    prob.solve(solver=cp.MOSEK, mosek_params = {mosek.dparam.optimizer_max_time: time_limit})
    x_value = [theta.value, y.value] # Combine y and theta into 1 single solution vector
    return (x_value, prob.value)

def unc_obj_func(x, data, **kwargs):
    # extract values
    C = kwargs['C']
    C_tilde = kwargs['C_tilde']
    U = kwargs['U']
    d,p = zip(*data)
    m,n = p[0].shape
    y = x[1]
    
    # compute obj function value:
    fixed_costs = np.sum(np.multiply(C, y))
    inv_cost = np.array([sum(C_tilde[k] * np.maximum(sum(p[s][j][k]*y[j][k] for j in range(m)) - d[s][k], 0)
                             for k in range(n)) for s in range(len(data))]) 
    rev = np.array([sum(U[k] * np.minimum(sum(p[s][j][k]*y[j][k] for j in range(m)),d[s][k]) 
                        for k in range(n)) for s in range(len(data))]) 
    return fixed_costs + inv_cost - rev

In [3]:
problem_instance = get_fixed_param_data()

In [4]:
# classic approach:
random_seed = 0
N = 10 #10580
data = generate_unc_param_data(random_seed, N)

start_time = time.time()
x, obj = solve_P_SCP(data, **problem_instance)
runtime = time.time() - start_time

print(obj)
print(runtime)

-518.852148045705
1.1807920932769775


In [5]:
# Care (2014) approach:
...

In [6]:
# generate and split data into train and test
random_seed = 0
N_total = 10000
data = generate_unc_param_data(random_seed, N_total)

N_train = N_total / 2
data_train, data_test = train_test_split(data, train_size=(N_train/N_total), random_state=random_seed)

In [7]:
# set our own algorithm parameter values
conf_param_alpha = 1e-9
add_strategy = 'random_vio'
remove_strategy = 'random_any'

In [8]:
# provide functions and other info for generating & evaluating solutions
solve_SCP = solve_P_SCP
eval_unc_obj = {'function': unc_obj_func,
                'info': {'risk_measure': 'probability', # must be either 'probability' or 'expectation'
                         'desired_rhs': 1 - 0.01}}

eval_unc_constr = None

In [9]:
# run the algorithm
alg = iter_gen_and_eval_alg(solve_SCP, problem_instance, eval_unc_obj, eval_unc_constr, 
                            data_train, data_test, conf_param_alpha=conf_param_alpha,
                            verbose=False)


stop_criteria={'max_elapsed_time': 1*60} # in seconds (time provided to search algorithm)

(best_sol, runtime, num_iter, pareto_frontier, S_history) = alg.run(stop_criteria=stop_criteria)

In [10]:
best_sol

{'sol': [array([-635.77057336]),
  array([[1.77984986e-11, 1.77533338e-11, 2.40879699e-11, 3.72504326e+00,
          2.77429719e+00, 1.46399705e-11, 3.50065955e+00, 1.85680345e-11,
          1.59725455e-09, 4.34314682e-11],
         [1.56638363e-11, 5.39364597e-11, 4.82729071e-09, 1.13502838e-09,
          2.65155885e+00, 2.75375886e+00, 4.45576246e-06, 7.83584907e-11,
          6.32383260e+00, 1.27084523e+00],
         [1.62898020e-10, 6.73283649e+00, 3.71593469e-09, 3.02832789e+00,
          9.95662666e-10, 5.67611395e+00, 3.23676351e-11, 9.56122337e-09,
          3.93390652e-11, 1.10043618e-10],
         [2.28763594e-11, 4.28217915e-11, 9.37685766e+00, 3.92086060e-11,
          5.77456749e-11, 4.40162787e-09, 2.47747368e+00, 7.14566865e+00,
          2.38476090e-11, 7.30249084e-10],
         [1.00391446e+01, 3.39837007e+00, 1.60896125e-04, 4.17319874e-11,
          3.12469302e-11, 3.02383295e-10, 2.26476457e-11, 5.10917804e-01,
          1.44585327e-11, 7.05140661e+00]])],
 'obj': -